In [1]:
pip install git+https://github.com/huggingface/transformers #Install Prerequisites

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-fhnmqbqw
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-fhnmqbqw

  Resolved https://github.com/huggingface/transformers to commit 52a02137557963e9dd58c9be65b6cef871d3bf32
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.5/776.5 kB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 179.0 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.45.0.dev0-py3-none-any.whl size=9616263 sha256=c7cd7246728100e8db24ac58714a79d5c79a0d273a023b7d1e9926d1708c5e3f
  Stored in directory: /tmp/pip-ephem-wheel-cache-bly7kka7/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
Note: you may need to restart the kernel to use updated pac

#Install Prerequisites

In [7]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 170.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 178.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 6.8 MB/s eta 0:00:00eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 165.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [7]:
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
from transformers import AutoModelForCausalLM
import torch
from transformers.trainer_callback import TrainerCallback
import os
from transformers import BitsAndBytesConfig
from trl import SFTTra

ImportError: cannot import name 'top_k_top_p_filtering' from 'transformers' (/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/__init__.py)

In [28]:
pip install accelerate==0.26.0

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.21.0
    Uninstalling accelerate-0.21.0:
      Successfully uninstalled accelerate-0.21.0
Note: you may need to restart the kernel to use updated packages.


Code for Model Inference

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer


def generate_response(model_name, input_text, max_new_tokens=50):
  
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    
  
    input_ids = tokenizer(input_text, return_tensors='pt').input_ids
    
   
    with torch.no_grad():
        generated_ids = model.generate(input_ids, max_new_tokens=max_new_tokens)
    

    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    
    return generated_text

if __name__ == "__main__":
    # Set the model name from Hugging Face Hub
    model_name = "AINovice2005/ElEmperador"  # Example model, you can change this to any other model
    input_text = "Hello, how are you?"

   
    output = generate_response(model_name, input_text)
    
    
    print(f"Input: {input_text}")
    print(f"Output: {output}")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of MistralForCausalLM were not initialized from the model checkpoint at AINovice2005/ElEmperador and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Input: Hello, how are you?
Output: Hello, how are you?ournalournalesiesiesiesiesiesiournalournalournalournalournalournalournalournalournalournalournalournalournalournalournalournalournalournalournalournalournalournalournalournalournalournalournalournalournalournalournalournalournalournalournalournalournalournalournalournalournalournal


In [1]:
from transformers import TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType
from transformers import AutoModelForCausalLM, AutoTokenizer

def generate_lora_configs(base_model_name, task_type=TaskType.CAUSAL_LM):
    configs = {}
    
    for rank in range(8, 33, 8):
        config = LoraConfig(
            r=rank,
            lora_alpha=16,
            target_modules=['q_proj','k_proj','v_proj','o_proj','gate_proj','down_proj','up_proj','lm_head'],
            lora_dropout=0.05,
            bias="none",
            task_type=task_type
        )
        configs[rank] = config
    
    return configs

def main():
    base_model_name = "AINovice2005/ElEmperador"  # You can change this to any other base model
    
    # Generate LoRA configs
    lora_configs = generate_lora_configs(base_model_name)
    
    # Load the base model and tokenizer
    model = AutoModelForCausalLM.from_pretrained(base_model_name)
    tokenizer = AutoTokenizer.from_pretrained(base_model_name)
    
    # Print configurations and create PEFT models
    for rank, config in lora_configs.items():
        print(f"\nLoRA Configuration for rank {rank}:")
        print(config)
        
        # Create PEFT model
        peft_model = get_peft_model(model, config)
        print(f"PEFT model created for rank {rank}")
        
        # You can add your training loop here if needed

if __name__ == "__main__":
    main()

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of MistralForCausalLM were not initialized from the model checkpoint at AINovice2005/ElEmperador and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



LoRA Configuration for rank 8:
LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'down_proj', 'up_proj', 'lm_head'], lora_alpha=16, lora_dropout=0.05, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None)
PEFT model created for rank 8

LoRA Configuration for rank 16:
LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=16, target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'down_proj', 'up_proj', 'lm_head'], lora_alpha=16, lora_dropout=0.05, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None)
PEFT m

Training Script

In [28]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model

# Step 1: Load the Pre-trained Model and Tokenizer
model_checkpoint = "AINovice2005/ElEmperador"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

# Step 2: Load Your Custom Dataset from a Folder Containing .arrow Files
dataset_dir = '/teamspace/studios/this_studio/path_to_save_tokenized_data1'
dataset = load_dataset(dataset_dir)

# Step 3: Preprocess the dataset to include input_ids
def preprocess_function(examples):
    # Tokenize the inputs and targets
    inputs = tokenizer(examples['_data_files_'], max_length=512, truncation=True, padding="max_length")
    targets = tokenizer(examples['response'], max_length=512, truncation=True, padding="max_length")

    # Prepare the model inputs
    model_inputs = {
        'input_ids': inputs['input_ids'],
        'attention_mask': inputs['attention_mask'],
        'labels': targets['input_ids']  # Labels must match the input_ids of the target sequence
    }
    return model_inputs

# Apply preprocessing to the entire dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Step 4: Apply LoRA Fine-Tuning
lora_config = LoraConfig(
    task_type="CASUAL_LM", 
    r=16,                             
    lora_alpha=32,                    
    lora_dropout=0.05,                
    target_modules=['q_proj','k_proj','v_proj','o_proj','gate_proj','down_proj','up_proj','lm_head']
)

# Wrap the model with LoRA using PEFT library
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# Step 5: Define Training Arguments
training_args = TrainingArguments(
    output_dir="./results",             
    evaluation_strategy="epoch",        
    learning_rate=5e-5,                 
    per_device_train_batch_size=4,      
    per_device_eval_batch_size=4,       
    weight_decay=0.01,                  
    save_total_limit=2,                 
    num_train_epochs=1,                 
    fp16=True,                          
    logging_steps=50,                   
    save_steps=100,                     
    gradient_accumulation_steps=8,      
    max_grad_norm=1.0,                  
    optim="adamw_torch",                
    max_steps=500,                      
    warmup_ratio=0.1,                   
    group_by_length=True,               
    lr_scheduler_type="linear",         
    report_to="tensorboard", 
    remove_unused_columns=False         
)

# Step 6: Initialize Trainer
trainer = Trainer(
    model=model,                        
    args=training_args,                 
    train_dataset=tokenized_dataset['train'],  # Use the preprocessed training dataset     
    tokenizer=tokenizer                
)

# Now you're ready to train the model
trainer.train()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of MistralForCausalLM were not initialized from the model checkpoint at AINovice2005/ElEmperador and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

KeyError: '_data_files_'